In [1]:
from pyarrow_item_schemas import item_schema, list_of_items_schema, item, id, version

In [7]:
item_schema

id: string not null
version: string not null

In [8]:
list_of_items_schema

items: list<item: struct<id: string not null, version: string not null>> not null
  child 0, item: struct<id: string not null, version: string not null>
      child 0, id: string not null
      child 1, version: string not null

In [37]:
from pydantic import BaseModel
class Item(BaseModel):
    id: str
    version: str

raw_items = [
    Item(id=str(i), version="1")
    for i in range(4)
]
raw_items

[Item(id='0', version='1'),
 Item(id='1', version='1'),
 Item(id='2', version='1'),
 Item(id='3', version='1')]

In [44]:
df1 = pd.DataFrame([item.model_dump() for item in raw_items])
df1

,id,version
0,0,1
1,1,1
2,2,1
3,3,1


In [45]:
items1 = [item.model_dump() for item in raw_items][:2]
items2 = [item.model_dump() for item in raw_items][2:]
items1, items2

([{'id': '0', 'version': '1'}, {'id': '1', 'version': '1'}],
 [{'id': '2', 'version': '1'}, {'id': '3', 'version': '1'}])

In [46]:
import pandas as pd
df2 = pd.DataFrame(data={
    "items": [items1, items2]
})
df2

,items
0,"[{'id': '0', 'version': '1'}, {'id': '1', 'ver..."
1,"[{'id': '2', 'version': '1'}, {'id': '3', 'ver..."


In [47]:
list_of_items_schema

items: list<item: struct<id: string not null, version: string not null>> not null
  child 0, item: struct<id: string not null, version: string not null>
      child 0, id: string not null
      child 1, version: string not null

In [49]:
pa.Table.from_pandas(df1, schema=item_schema)

pyarrow.Table
id: string not null
version: string not null
----
id: [["0","1","2","3"]]
version: [["1","1","1","1"]]

In [50]:
pa.Table.from_pandas(df2, schema=list_of_items_schema)

pyarrow.Table
items: list<item: struct<id: string not null, version: string not null>> not null
  child 0, item: struct<id: string not null, version: string not null>
      child 0, id: string not null
      child 1, version: string not null
----
items: [[    -- is_valid: all not null
    -- child 0 type: string
["0","1"]
    -- child 1 type: string
["1","1"],    -- is_valid: all not null
    -- child 0 type: string
["2","3"]
    -- child 1 type: string
["1","1"]]]

In [54]:
df2

,items
0,"[{'id': '0', 'version': '1'}, {'id': '1', 'ver..."
1,"[{'id': '2', 'version': '1'}, {'id': '3', 'ver..."


In [55]:
df2_copy = df2.copy(deep=True)
df2_copy

,items
0,"[{'id': '0', 'version': '1'}, {'id': '1', 'ver..."
1,"[{'id': '2', 'version': '1'}, {'id': '3', 'ver..."


In [58]:
df2_copy['objects'] = df2_copy.apply(lambda row: [Item.model_validate(i) for i in row['items']], axis=1)
df2_copy

,items,objects
0,"[{'id': '0', 'version': '1'}, {'id': '1', 'ver...","[id='0' version='1', id='1' version='1']"
1,"[{'id': '2', 'version': '1'}, {'id': '3', 'ver...","[id='2' version='1', id='3' version='1']"


In [60]:
df2_copy.iloc[0]['objects'][0]

Item(id='0', version='1')